주피터를 실행하자마자 에러가 떴는데, pip를 새로 설치하면서 jupyter가 pip에 없어서 발생하는 에러였다!  
당황하지 말고 pip install jupyter로 해결해주자.

In [1]:
# 진행률을 표기하기 위한 tqdm
from tqdm import tqdm

# DataFrame분석을 위한 pandas
import pandas as pd

# validation을 위한 전처리 train_test_split
from sklearn.model_selection import train_test_split

# 분석 및 평가를 위한 sklearn.metrics
from sklearn.metrics import confusion_matrix, accuracy_score

# 시각화 모듈을 위한 matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn

## 1. Read Datasets

In [4]:
# 용량이 상당히 크다... (무려 2.7기가)
# train = pd.read_csv("./datasets/train.csv")
# test = pd.read_csv("./datasets/test.csv")
# print(len(train))

# 데이터 양이 애초에 너무 많아, 샘플로 깎아서 진행한다.
# train_df_sample = train.sample(frac=0.1, random_state=1004)


28605391


In [ ]:
## 그래서 아래 10분할 해서 train_df-_sample만 사용하기로 했다
train_df_sample = pd.read_csv("./train_df_sample.csv")

In [7]:
len(test)

4538541

In [8]:
len(train_df_sample)

2860539

In [9]:
train_df_sample.head()

,ID,Click,F01,F02,F03,F04,F05,F06,F07,F08,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
27047686,TRAIN_27047686,0,CYOAMVC,DJDKEYH,NaN,NaN,XCAJWBW,13,VNXTVLH,VAWXMCR,...,TFZIBRI,GTISJWW,9787.0,0.0,NPEGYAH,IRUDRFB,15.0,JSOMQYE,0.0,KOFRDGL
7223697,TRAIN_07223697,0,JCDXFYU,PILDDJU,IAGJDOH,1.0,LFPUEOV,68,MHBXSWB,LPYPUNA,...,REPSWLB,KHZNEZF,15063.0,0.0,QMOULXS,IRUDRFB,NaN,XBKBHCW,0.0,LNUQAZZ
21459435,TRAIN_21459435,0,VNOHLIR,PKLDGLX,IAGJDOH,5.0,EVTUBMN,6,JNIVDXP,LPYPUNA,...,SLXYBBG,GTISJWW,601.0,0.0,SHMKPOR,IRUDRFB,6.0,FXWZZCX,0.0,XHYNPHU
6264302,TRAIN_06264302,0,UCFAVXY,DJDKEYH,NaN,3.0,WNMKDBA,1,YCWUGFD,VAWXMCR,...,MFPUCBU,ENBEWZP,4210.0,0.0,DVESEGJ,IRUDRFB,3.0,VDQUXYS,0.0,JDVFUQP
22182054,TRAIN_22182054,0,JCDXFYU,PILDDJU,IAGJDOH,11.0,LFPUEOV,81,PIFJCMX,FTPHMPQ,...,NZGEZLW,KHZNEZF,1.0,39.0,QMOULXS,IRUDRFB,0.0,AQGCGGG,1.0,QHKAWMA


In [10]:
# sample 만들기.
train_df_sample.to_csv("./train_df_sample.csv",
                       sep=',')

In [11]:
train_df_sample.columns

Index(['ID', 'Click', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18',
       'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28',
       'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38',
       'F39'],
      dtype='object')

In [12]:
train_df_sample.isna()

,ID,Click,F01,F02,F03,F04,F05,F06,F07,F08,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
27047686,False,False,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7223697,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
21459435,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6264302,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22182054,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10990539,False,False,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
7434772,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19701460,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19404996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


데이터를 조금 살펴보면, 결국 우리가 알아야 하는 것은 정보를 종합하여 봤을 때 `click` 을 하느냐 마느냐.

> train_y = 'Click'
> train_x = ['ID', 'Click']을 제외한 나머지

In [13]:
"""
Dataset Info.

train.csv [파일] 
시간 순으로 나열된 7일 동안의 웹 광고 클릭 로그
ID: train 데이터 샘플 고유 ID
Click: 예측 목표인 클릭 여부
0: 클릭하지 않음, 1: 클릭
F01 ~ F39 : 각 클릭 로그와 연관된 Feature
개인정보 보호를 위해 상세 정보는 비식별 처리됨

"""

train_df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2860539 entries, 27047686 to 27190539
Data columns (total 41 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ID      object 
 1   Click   int64  
 2   F01     object 
 3   F02     object 
 4   F03     object 
 5   F04     float64
 6   F05     object 
 7   F06     int64  
 8   F07     object 
 9   F08     object 
 10  F09     object 
 11  F10     object 
 12  F11     float64
 13  F12     object 
 14  F13     object 
 15  F14     int64  
 16  F15     object 
 17  F16     object 
 18  F17     object 
 19  F18     float64
 20  F19     float64
 21  F20     object 
 22  F21     object 
 23  F22     object 
 24  F23     object 
 25  F24     float64
 26  F25     object 
 27  F26     object 
 28  F27     float64
 29  F28     object 
 30  F29     float64
 31  F30     object 
 32  F31     object 
 33  F32     float64
 34  F33     float64
 35  F34     object 
 36  F35     object 
 37  F36     float64
 38  F37     object 
 39  F38     float

In [18]:
train_object_col = train_df_sample.select_dtypes(['object'])
train_numeric_col = train_df_sample.select_

In [14]:
train_df_sample.describe()

,Click,F04,F06,F11,F14,F18,F19,F24,F27,F29,F32,F33,F36,F38
count,2.860539e+06,2.286398e+06,2.860539e+06,2.564728e+06,2.860539e+06,2.127584e+06,2.601387e+06,1.962540e+06,1.754219e+06,1.754219e+06,2.835559e+06,2.601387e+06,2.127584e+06,2.780468e+06
mean,1.943284e-01,3.121925e+01,1.175535e+02,4.063136e+02,9.613142e+00,6.838256e+00,3.433157e-01,1.139983e+02,2.538591e+01,4.529029e+00,1.943406e+04,2.021605e+00,8.156234e+00,1.854519e-01
std,3.956829e-01,4.534102e+02,4.039554e+02,6.634069e+02,1.469758e+01,8.967519e+00,6.112556e-01,5.882291e+02,8.971299e+01,7.833788e+00,7.202122e+04,3.276068e+01,1.873038e+01,1.999053e+00
min,0.000000e+00,1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,3.000000e+00,1.000000e+00,5.700000e+01,1.000000e+00,2.000000e+00,0.000000e+00,7.000000e+00,2.000000e+00,1.000000e+00,3.540000e+02,0.000000e+00,2.000000e+00,0.000000e+00
50%,0.000000e+00,7.000000e+00,5.000000e+00,1.830000e+02,5.000000e+00,4.000000e+00,0.000000e+00,2.700000e+01,6.000000e+00,2.000000e+00,2.711000e+03,0.000000e+00,4.000000e+00,0.000000e+00
75%,0.000000e+00,2.100000e+01,5.000000e+01,4.850000e+02,1.400000e+01,9.000000e+00,1.000000e+00,8.200000e+01,1.800000e+01,4.000000e+00,1.022100e+04,1.000000e+00,1.000000e+01,0.000000e+00
max,1.000000e+00,6.553500e+04,2.687100e+04,8.000000e+03,7.382000e+03,9.270000e+02,1.000000e+01,3.220220e+05,7.546000e+03,2.200000e+02,2.626120e+06,5.132400e+04,1.211200e+04,6.260000e+02
